In [ ]:
%reset -f 

In [ ]:
# convert NHD waterbody from GDB to Polygon
arcpy.management.FeatureToPolygon(r"NHD\NHDWaterbody", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\NHDWaterbody_FeatureToPolygo", None, "ATTRIBUTES", None)

In [ ]:
# reproject watershed boundaries to NHD projection
arcpy.management.Project("AQSOLE_POLY", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\AQSOLE_POLY", 'GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["NAVD_1988",VDATUM["North_American_Vertical_Datum_1988"],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', None, 'PROJCS["NAD_1983_StatePlane_Massachusetts_Mainland_FIPS_2001",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",200000.0],PARAMETER["False_Northing",750000.0],PARAMETER["Central_Meridian",-71.5],PARAMETER["Standard_Parallel_1",41.71666666666667],PARAMETER["Standard_Parallel_2",42.68333333333333],PARAMETER["Latitude_Of_Origin",41.0],UNIT["Meter",1.0]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

In [ ]:
# clip land use for aquifer extent
arcpy.analysis.Clip("Land Cover Land Use (2016)", "AQSOLE_POLY", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\LandCoverLandUse2016_Clip", None)

In [ ]:
# zonal stats to calc 5th percentile of elevation in each subcatchment
out_raster = arcpy.ia.ZonalStatistics("agawam_subw_0309", "SubWat_Num", "Extract_LiDA1", "PERCENTILE", "DATA", "CURRENT_SLICE", 5, "AUTO_DETECT"); out_raster.save(r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\ZonalSt_agaw1")

In [1]:
# zonal stats raster output to polygon 
arcpy.conversion.RasterToPolygon("extract_lida_le5pct", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\RasterT_extract1", "SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

<Result 'C:\\Workspace\\Geodata\\Groundwater\\Groundwater.gdb\\RasterT_extract1'>

In [2]:
# dissolve polygon
arcpy.management.Dissolve("RasterT_extract1", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\RasterT_extract1_Dissolve", "gridcode", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'C:\\Workspace\\Geodata\\Groundwater\\Groundwater.gdb\\RasterT_extract1_Dissolve'>

In [3]:
# intersect zonal stats binary polygon with watershed boundaries
arcpy.analysis.Intersect("RasterT_extract1_Dissolve #;agawam_subw_0309_Dissolve #", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\Agawam_Terminus", "ALL", None, "INPUT")

In [ ]:
# dissolve subs
arcpy.management.Dissolve("agawam_subw_0309", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\agawam_subw_0309_Dissolve", "SubWat_Num;SubW_Names", None, "MULTI_PART", "DISSOLVE_LINES")

In [1]:
# intersect landcover class polygons with terminus and subwatershed layer. 
arcpy.analysis.Identity("out_tidmarsh_lclu16_Identity", "Agawam_Terminus", r"C:\Workspace\Geodata\Groundwater\Groundwater.gdb\Agawam_Terminus_lclu16_Identity", "ALL", None, "NO_RELATIONSHIPS")

<Result 'C:\\Workspace\\Geodata\\Groundwater\\Groundwater.gdb\\Agawam_Terminus_lclu16_Identity'>

In [98]:
import pandas as pd
import os
import pickle

In [128]:
df_raw = pd.read_pickle("df_Agawam_Terminus_lclu16_Identity_TableToExcel.pkl")

In [129]:
df_raw.shape

(114650, 61)

In [130]:
df_raw

,OBJECTID,Join_Count,TARGET_FID,FID_out_ag,FID_LANDCO,COVERNAME,COVERCODE,USEGENNAME,USEGENCODE,USE_CODE,POLY_TYPE,FY,TOWN_ID,TILENAME,Shape_Leng,FID_agawam,sf_area,SubWat_Num,New_ID,ft_perim,SubW_Names,SHED_ID,Terminal_N,F_,Area_Ha,FID_in_war,WMA_NO,OWNER,ADDRESS,TOWN,REGION,BOG_NAME,STAFF,PROGRAM,DATE_ENTER,COMMENT,BIRTHREG,PERMIT_NUM,CropStatus,Bog_Code,Bog_Name_2,Active,Plant_Year,Variety,Var_Class,Bog_type,Area_sqft,Area_ha_1,renovated,ren_yr,new_upland,new_yr,New_ID_1,SubW_Nam_1,Basin_1,BOG_FID,FID_RasterT_extract1_Dissolve,Terminus,FID_agawam_subw_0309_Dissolve,Shape_Length,Shape_Area
0,1,1,0,0,17132259,Impervious,2,Unknown,0,000,FEE,2016,310,R12C23,358.492424,34,2.475554e+07,40,40,28134.229416,Parker Mills Pond,40,p,0,0.098950,-1,,,,,0,,,,00:00:00,,,,,,,,,,,,0.0,0.00000,,,,,0,,,,1.0,0.0,40.0,35111.016660,989.500000
1,2,2,1,1,17132780,Grassland,8,Unknown,0,000,FEE,2016,310,R12C23,118.209868,34,2.475554e+07,40,40,28134.229416,Parker Mills Pond,40,p,0,0.009283,-1,,,,,0,,,,00:00:00,,,,,,,,,,,,0.0,0.00000,,,,,0,,,,1.0,0.0,39.0,30195.107169,185.792112
2,3,2,2,2,17132840,Evergreen Forest,10,Agriculture,7,7200,FEE,2016,310,R12C23,290.392311,34,2.475554e+07,40,40,28134.229416,Parker Mills Pond,40,p,0,0.161413,-1,,,,,0,,,,00:00:00,,,,,,,,,,,,0.0,0.00000,,,,,0,,,,1.0,0.0,40.0,35111.016660,1615.104314
3,4,1,3,3,17132841,Evergreen Forest,10,Agriculture,7,7200,FEE,2016,310,R12C23,2.884870,34,2.475554e+07,40,40,28134.229416,Parker Mills Pond,40,p,0,0.000016,-1,,,,,0,,,,00:00:00,,,,,,,,,,,,0.0,0.00000,,,,,0,,,,1.0,0.0,40.0,35111.016660,0.161639
4,5,1,4,4,17132842,Evergreen Forest,10,Agriculture,7,7200,FEE,2016,310,R12C23,47.419592,34,2.475554e+07,40,40,28134.229416,Parker Mills Pond,40,p,0,0.003391,-1,,,,,0,,,,00:00:00,,,,,,,,,,,,0.0,0.00000,,,,,0,,,,1.0,0.0,40.0,35111.016660,33.906300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114645,114646,1,114645,97654,17300857,Grassland,8,Agriculture,7,0761,TAX,2016,239,R12C24,645.946789,37,3.800523e+07,33,33,36179.658437,Frogfoot Brook,33,s,0,0.001241,241,42431033,A.D. Makepeace Co.,Tihonet Rd.,Plymouth,4,Frog Foot Main,Jim McLaughlin,BRP_WMA,2007-06-18 00:00:00,point 5 (70.72),42431033,9P242431003,active,FR34,,,,EB & H,,flowthrough,679657.0,6.31425,,,,,33,Frogfoot Brook,Wankinco,241,2.0,1.0,34.0,12418.977916,3629.625000
114646,114647,1,114646,97771,17301726,Deciduous Forest,9,Agriculture,7,0761,TAX,2016,239,R12C24,121.932504,37,3.800523e+07,33,33,36179.658437,Frogfoot Brook,33,s,0,0.008807,241,42431033,A.D. Makepeace Co.,Tihonet Rd.,Plymouth,4,Frog Foot Main,Jim McLaughlin,BRP_WMA,2007-06-18 00:00:00,point 5 (70.72),42431033,9P242431003,active,FR34,,,,EB & H,,flowthrough,679657.0,6.31425,,,,,33,Frogfoot Brook,Wankinco,241,2.0,1.0,34.0,12418.977916,554.625000
114647,114648,1,114647,97904,17302602,Palustrine Forested Wetland,13,Agriculture,7,0761,TAX,2016,239,R12C24,269.178716,37,3.800523e+07,33,33,36179.658437,Frogfoot Brook,33,s,0,0.012413,241,42431033,A.D. Makepeace Co.,Tihonet Rd.,Plymouth,4,Frog Foot Main,Jim McLaughlin,BRP_WMA,2007-06-18 00:00:00,point 5 (70.72),42431033,9P242431003,active,FR34,,,,EB & H,,flowthrough,679657.0,6.31425,,,,,33,Frogfoot Brook,Wankinco,241,2.0,1.0,34.0,12418.977916,1873.000000
114648,114649,1,114648,109625,17296893,Developed Open Space,5,Tax exempt,9,9570,TAX,2016,310,R12C24,27.749647,32,2.052870e+08,20,20,130795.386117,Mill Pond,20,p,0,0.004133,-1,,,,,0,,,,00:00:00,,,,,,,,,,,,0.0,0.00000,,,,,0,,,,1.0,0.0,21.0,171785.350482,41.325427


In [131]:
df_raw.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'FID_out_ag', 'FID_LANDCO',
       'COVERNAME', 'COVERCODE', 'USEGENNAME', 'USEGENCODE', 'USE_CODE',
       'POLY_TYPE', 'FY', 'TOWN_ID', 'TILENAME', 'Shape_Leng', 'FID_agawam',
       'sf_area', 'SubWat_Num', 'New_ID', 'ft_perim', 'SubW_Names', 'SHED_ID',
       'Terminal_N', 'F_', 'Area_Ha', 'FID_in_war', 'WMA_NO', 'OWNER',
       'ADDRESS', 'TOWN', 'REGION', 'BOG_NAME', 'STAFF', 'PROGRAM',
       'DATE_ENTER', 'COMMENT', 'BIRTHREG', 'PERMIT_NUM', 'CropStatus',
       'Bog_Code', 'Bog_Name_2', 'Active', 'Plant_Year', 'Variety',
       'Var_Class', 'Bog_type', 'Area_sqft', 'Area_ha_1', 'renovated',
       'ren_yr', 'new_upland', 'new_yr', 'New_ID_1', 'SubW_Nam_1', 'Basin_1',
       'BOG_FID', 'FID_RasterT_extract1_Dissolve', 'Terminus',
       'FID_agawam_subw_0309_Dissolve', 'Shape_Length', 'Shape_Area'],
      dtype='object')

In [132]:
# make a copy and work on selected columns. 
df_raw_copy = df_raw.copy()
df_raw_copy is df_raw

False

In [133]:
df = df_raw_copy[['COVERNAME','COVERCODE','USEGENNAME','USEGENCODE','USE_CODE','SubW_Names','SHED_ID','Terminal_N','Terminus','CropStatus','BOG_FID','Shape_Length','Shape_Area']]

In [134]:
df.columns

Index(['COVERNAME', 'COVERCODE', 'USEGENNAME', 'USEGENCODE', 'USE_CODE',
       'SubW_Names', 'SHED_ID', 'Terminal_N', 'Terminus', 'CropStatus',
       'BOG_FID', 'Shape_Length', 'Shape_Area'],
      dtype='object')

In [135]:
df = df.rename(columns={"Terminus": "In_Terminus", "Terminal_N": "Old_Terminus_Class"})

In [118]:
# use one hot encoding for categorical variables. 
df = pd.get_dummies(df,columns=['COVERNAME','USEGENNAME','CropStatus'],drop_first=True)

In [136]:
df.columns

Index(['COVERNAME', 'COVERCODE', 'USEGENNAME', 'USEGENCODE', 'USE_CODE',
       'SubW_Names', 'SHED_ID', 'Old_Terminus_Class', 'In_Terminus',
       'CropStatus', 'BOG_FID', 'Shape_Length', 'Shape_Area'],
      dtype='object')

In [143]:
df_grouped_sum = df.groupby(['SubW_Names','In_Terminus','COVERNAME','USEGENNAME',]).sum()
df_grouped_sum

COVERCODE  ...    Shape_Area
SubW_Names In_Terminus COVERNAME                      USEGENNAME                              ...              
Abner Pond 0.0         Bare Land                      Residential - single family         40  ...  4.835000e+02
                                                      Tax exempt                          20  ...  6.548750e+02
                       Deciduous Forest               Residential - single family        360  ...  2.298913e+04
                                                      Tax exempt                         306  ...  2.997300e+04
                       Developed Open Space           Residential - single family         70  ...  1.117866e+03
...                                                                                      ...  ...           ...
bog stream 1.0         Palustrine Scrub/Shrub Wetland Open land                           70  ...  1.847660e+03
                                                      Residential - multi-family          14  ...  5.127235e+02
                                                      Residential - single family         42  ...  3.252893e+01
                       Water                          Open land                           84  ...  2.683258e+03
                                                      Unknown                             42  ...  1.079545e+06

[3161 rows x 5 columns]

In [142]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

sns.set_theme(style="ticks")

'''
# Create a dataset with many short random walks
rs = np.random.RandomState(4)
pos = rs.randint(-1, 2, (20, 5)).cumsum(axis=1)
pos -= pos[:, 0, np.newaxis]
step = np.tile(range(5), 20)
walk = np.repeat(range(20), 5)
df = pd.DataFrame(np.c_[pos.flat, step, walk],
                  columns=["position", "step", "walk"])
'''

# Initialize a grid of plots with an Axes for each walk
grid = sns.FacetGrid(df_grouped_sum, col="SubW_Names", hue="In_Terminus", palette="tab20c",
                     col_wrap=4, height=1.5)


# Draw a horizontal line to show the starting point
grid.refline(y=0, linestyle=":")

# Draw a line plot to show the trajectory of each random walk
grid.map(sns.catplot, "COVERCODE", "Shape_Area", marker="o")

## Adjust the tick positions and labels
#grid.set(xticks=np.arange(5), yticks=[-3, 3],
#         xlim=(-.5, 4.5), ylim=(-3.5, 3.5))

# Adjust the arrangement of the plots
#grid.fig.tight_layout(w_pad=1)

plt.show()

KeyError: 'In_Terminus'

<module 'matplotlib.pyplot' from 'C:\\Users\\Adrian.Wiegman\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\matplotlib\\pyplot.py'>